In [59]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from tensorflow.keras.layers import Embedding, Flatten, LSTM, Dense, Input, Concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import Input



ImportError: cannot import name 'ReShape' from 'keras.layers' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/layers/__init__.py)

In [2]:
# Load the data
data_path = "../data/Eye-tracking Output/cleaned_data.csv"
df = pd.read_csv(data_path)


In [31]:
## Normalizing data ##

# Feature selection of relevant columns
relevant_columns = ['Point of Regard Right X [px]', 'Point of Regard Right Y [px]',
                    'Pupil Diameter Left [mm]', 'Pupil Diameter Right [mm]', 'Tracking Ratio [%]',
                    'Category Right', 'Participant', 'Trial', 'Stimulus', 'Gender', 'Age', 'Class', 'CARS Score']
df_relevant = df[relevant_columns]

# Filling NaNs in 'CARS Score' with 0
df_relevant['CARS Score'].fillna(0, inplace=True)
df_relevant['Age'].fillna(0, inplace=True)

# List of numerical columns to scale
numerical_columns = ['Point of Regard Right X [px]', 'Point of Regard Right Y [px]',
                     'Pupil Diameter Left [mm]', 'Pupil Diameter Right [mm]'] # Lidt i tvivl om vi skal have 'Age' med her. CARS og Age giver også 0. Pis lort

# Convert columns to numeric, coercing errors to NaN
for col in numerical_columns:
    df_relevant[col] = pd.to_numeric(df_relevant[col], errors='coerce')

# Define a function to fill NaN with the mean of the previous and next row
def fill_with_row_mean(df_relevant, col):
    # First, forward fill the first NaN (if any)
    df_relevant[col] = df_relevant[col].fillna(method='ffill')
    
    # Then, fill the rest with the mean of the previous and next row
    df_relevant[col] = df_relevant[col].fillna((df_relevant[col].shift(1) + df_relevant[col].shift(-1)) / 2)
    
    return df_relevant[col]

# Apply this function to each numerical column
for col in numerical_columns:
    df_relevant[col] = fill_with_row_mean(df_relevant, col)

# Handle any remaining NaNs, especially at the end of the DataFrame
df_relevant.fillna(method='bfill', inplace=True)

# Normalize data per combination of Trial, Participant, and Stimulus
for (trial, participant, stimulus), group_data in df_relevant.groupby(['Trial', 'Participant', 'Stimulus']):
    scaler = MinMaxScaler()
    # Apply the scaler to all numerical columns for this group
    df_relevant.loc[group_data.index, numerical_columns] = scaler.fit_transform(group_data[numerical_columns])

# For some reason Age and CARS Score are not scaled properly, so we do it manually
scaler = MinMaxScaler()
df_relevant[['CARS Score', 'Age', 'Tracking Ratio [%]']] = scaler.fit_transform(df_relevant[['CARS Score', 'Age', 'Tracking Ratio [%]']])

# Save the normalized data
df_relevant.to_csv


/var/folders/94/903y39c95x3g49g0_7l22jn00000gn/T/ipykernel_12081/479530185.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant['CARS Score'].fillna(0, inplace=True)
/var/folders/94/903y39c95x3g49g0_7l22jn00000gn/T/ipykernel_12081/479530185.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant['Age'].fillna(0, inplace=True)
/var/folders/94/903y39c95x3g49g0_7l22jn00000gn/T/ipykernel_12081/479530185.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [11]:
df_relevant = pd.read_csv("../data/Eye-tracking Output/normalized_data.csv")
# checking the variables to convert into dummy variables.
from sklearn.preprocessing import LabelEncoder

# for column in df_relevant.columns:
#     if df_relevant.dtypes[column] == "object":
#         print(column)

# print(df_relevant["Gender"].unique())
# print(df_relevant["Category Left"].unique())
# print(df_relevant["Category Right"].unique())
# print(df_relevant["Trial"].unique())
print(df_relevant["Stimulus"])

lol = df_relevant["Stimulus"].unique()

print(lol.shape)


# print(df_relevant["Gender"].unique())
le_gen = LabelEncoder()
df_relevant['Gender'] = le_gen.fit_transform(df_relevant['Gender']) # M, F




le_class = LabelEncoder()
df_relevant['Class'] = le_class.fit_transform(df_relevant['Class']) # ASD, TD


# Assuming 'Category' is your categorical variable in a DataFrame df
df_encoded = pd.get_dummies(df_relevant, columns=['Category Right', 'Gender', 'Class'], prefix=['Category Right', 'Gender', 'Class']) # one hot encoding

# print(df_encoded.iloc[:, 16:22].head(5)) # checking how it looks.




#print(df_relevant["Class"].unique())




0                 NoImage
1                 NoImage
2                 NoImage
3                 NoImage
4                 NoImage
               ...       
905514    18 aurevoir.jpg
905515    18 aurevoir.jpg
905516    18 aurevoir.jpg
905517    18 aurevoir.jpg
905518    18 aurevoir.jpg
Name: Stimulus, Length: 905519, dtype: object
(114,)


In [66]:
# Combining all data into a single tensor
# reshaping the normalized data into 3d np array
normalized_np = np.stack([df_normalized[col].values for col in df_normalized.columns], 1)
# converting from np array to keras tensors
normalized_tensor = tf.convert_to_tensor(normalized_np, dtype=tf.float32)

# Add a dimension to normalized_tensor and df_encoded
# This transforms them from shape (905519, 7) and (905519, 19) to (905519, 1, 7) and (905519, 1, 19)
normalized_tensor_3d = tf.expand_dims(normalized_tensor, axis=1)
df_encoded_3d = tf.expand_dims(df_encoded, axis=1)

# Now you can concatenate along the last axis
df_all = tf.keras.layers.Concatenate(axis=-1)([normalized_tensor_3d, df_encoded_3d, concatenated])



In [31]:
# Splitting the data into training and test sets

def create_sequences(data, sequence_length):
    sequences = []
    labels = []  # Adjust based on your task
    for i in range(len(data) - sequence_length):
        sequence = data[i:i + sequence_length]
        label = ...  # Define your label
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

sequence_length = 60  # 1 second of data - Er lidt usikker på denne her, da det ligner at de har et sample hvert 20. millisekund.
data_sequences, labels = create_sequences(df_relevant.values, sequence_length)

# Reshape for LSTM
X = data_sequences.reshape((data_sequences.shape[0], sequence_length, data_sequences.shape[2]))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, shuffle=False)



In [7]:
# print(X.shape)

indices = torch.randperm(len(X))[:200]

lort = X[indices] # making a subset of 200

# lort.shape 

In [ ]:

# Define the LSTM model
model = Sequential()
# Add LSTM layers and other necessary layers
# ...

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
# ...

# Evaluate the model
# ...

# Make predictions
# ...

NameError: name 'Sequential' is not defined